In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
import pandas as pd
from PIL import Image
import warnings

warnings.filterwarnings('ignore')

In [2]:
def get_first_digit(left_codes):
    parity_dict = {}
    parity_dict["OOOOOO"] = 0
    parity_dict["OOEOEE"] = 1
    parity_dict["OOEEOE"] = 2
    parity_dict["OOEEEO"] = 3
    parity_dict["OEOOEE"] = 4
    parity_dict["OEEOOE"] = 5
    parity_dict["OEEEOO"] = 6
    parity_dict["OEOEOE"] = 7
    parity_dict["OEOEEO"] = 8
    parity_dict["OEEOEO"] = 9
    parity = ""
    for code in left_codes:
        parity = parity + code["parity"]
    return parity_dict[parity]

In [3]:
def replace_255_to_1(array):
    for i in range(len(array)):
        if array[i] == 255:
            array[i] = 1

In [4]:
def get_ean13(left_codes,right_codes):
    ean13 = ""
    ean13 = ean13 + str(get_first_digit(left_codes))
    for code in left_codes:
        ean13 = ean13 + str(code["code"])
    for code in right_codes:
        ean13 = ean13 + str(code["code"])
    return ean13

In [5]:
def verify(ean13):
    weight = [1,3,1,3,1,3,1,3,1,3,1,3,1,3]
    weighted_sum = 0
    for i in range(12):
        weighted_sum = weighted_sum + weight[i] * int(ean13[i])
    weighted_sum = str(weighted_sum)
    checksum = 0
    units_digit = int(weighted_sum[-1])
    if units_digit != 0:
        checksum = 10 - units_digit
    else:
        checksum = 0

    if checksum == int(ean13[-1]):
        return True
    else:
        return False

In [6]:
def classify_bars(bars):
    left_guard = bars[0:3]
    left_patterns = bars[3:27]
    center_guard = bars[27:32]
    right_patterns = bars[32:56]
    right_guard = bars[56:59]
    return left_guard, left_patterns, center_guard, right_patterns, right_guard

In [7]:
def read_bars(line):
    replace_255_to_1(line)
    bars = []
    current_length = 1
    for i in range(len(line)-1):
        if line[i] == line[i+1]:
            current_length = current_length + 1
        else:
            bars.append(current_length * str(line[i]))
            current_length = 1
    bars.pop(0)
    return bars

In [8]:
def decode_right(at1, at2, m1, m2, m3, m4):
    patterns = {}
    patterns["2,2"]={"code":"6"}
    patterns["2,4"]={"code":"4"}
    patterns["3,3"]={"code":"8","alter_code":"2"}
    patterns["3,5"]={"code":"5"}
    patterns["4,2"]={"code":"9"}
    patterns["4,4"]={"code":"7","alter_code":"1"}
    patterns["5,3"]={"code":"0"}
    patterns["5,5"]={"code":"3"}
    pattern_dict = patterns[str(at1) + "," + str(at2)]
    code = 0
    use_alternative = False
    if int(at1) == 3 and int(at2) == 3:
        if m3+1>=m4:
            use_alternative = True
    if int(at1) == 4 and int(at2) == 4:
        if m1+1>=m2:
            use_alternative = True            
    if use_alternative:
        code = pattern_dict["alter_code"]
    else:
        code = pattern_dict["code"]
    final = {"code": code}
    return final

In [9]:
def decode_left(at1, at2, m1, m2, m3, m4):
    patterns = {}
    patterns["2,2"]={"code":"6","parity":"O"}
    patterns["2,3"]={"code":"0","parity":"E"}
    patterns["2,4"]={"code":"4","parity":"O"}
    patterns["2,5"]={"code":"3","parity":"E"}
    patterns["3,2"]={"code":"9","parity":"E"}
    patterns["3,3"]={"code":"8","parity":"O","alter_code":"2"}
    patterns["3,4"]={"code":"7","parity":"E","alter_code":"1"}
    patterns["3,5"]={"code":"5","parity":"O"}
    patterns["4,2"]={"code":"9","parity":"O"}
    patterns["4,3"]={"code":"8","parity":"E","alter_code":"2"}
    patterns["4,4"]={"code":"7","parity":"O","alter_code":"1"}
    patterns["4,5"]={"code":"5","parity":"E"}
    patterns["5,2"]={"code":"6","parity":"E"}
    patterns["5,3"]={"code":"0","parity":"O"}
    patterns["5,4"]={"code":"4","parity":"E"}
    patterns["5,5"]={"code":"3","parity":"O"}
    pattern_dict = patterns[str(at1) + "," + str(at2)]
    code = 0
    use_alternative = False
    if int(at1) == 3 and int(at2) == 3:
        if m3+1>=m4:
            use_alternative = True
    if int(at1) == 3 and int(at2) == 4:
        if m2+1>=m3:
            use_alternative = True
    if int(at1) == 4 and int(at2) == 3:
        if m2+1>=m1:
            use_alternative = True
    if int(at1) == 4 and int(at2) == 4:
        if m1+1>=m2:
            use_alternative = True            
    if use_alternative:
        code = pattern_dict["alter_code"]
    else:
        code = pattern_dict["code"]
    final = {"code": code, "parity": pattern_dict["parity"]}
    return final    

In [10]:
def get_AT(value):
    if value < 2.5/7:
        return 2
    elif value < 3.5/7:
        return 3
    elif value < 4.5/7:
        return 4
    else:
        return 5

In [11]:
def read_patterns(patterns,is_left=True):
    codes = []
    for i in range(6):
        start_index = i*4
        sliced = patterns[start_index:start_index+4]
        m1 = sliced[0]
        m2 = sliced[1]
        m3 = sliced[2]
        m4 = sliced[3]
        total = m1+m2+m3+m4
        tmp1=(m1+m2)*1.0
        tmp2=(m2+m3)*1.0
        at1 = get_AT(tmp1/total)
        at2 = get_AT(tmp2/total)
        if is_left:
            decoded = decode_left(at1,at2,m1,m2,m3,m4)
        else:
            decoded = decode_right(at1,at2,m1,m2,m3,m4)
        codes.append(decoded)
    return codes

In [12]:
def convert_patterns_to_length(patterns):
    for i in range(len(patterns)):
        patterns[i] = len(patterns[i])

In [13]:
def decode_line(line):
    bars = read_bars(line)
    left_guard, left_patterns, center_guard, right_patterns, right_guard = classify_bars(bars)
    convert_patterns_to_length(left_patterns)
    convert_patterns_to_length(right_patterns)
    left_codes = read_patterns(left_patterns,is_left=True)
    right_codes = read_patterns(right_patterns,is_left=False)
    ean13 = get_ean13(left_codes,right_codes)
    
    is_valid = verify(ean13)
    return ean13, is_valid

In [14]:
def decode(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh =cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    thresh = cv2.bitwise_not(thresh)
    ean13 = None
    is_valid = None
    for i in range(img.shape[0]-1,0,-1):
        try:
            ean13, is_valid = decode_line(thresh[i])
        except Exception as e:
            pass
        if is_valid:
            break
        
    return ean13, is_valid, thresh

In [15]:
def crop_rect(rect, box, img):
    W = rect[1][0]
    H = rect[1][1]
    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)
     
    
    center = ((x1+x2)/2,(y1+y2)/2)
    size = (x2-x1, y2-y1)
    cropped = cv2.getRectSubPix(img, size, center)
    
    
    angle = rect[2]
    if angle!=90:
        if angle>45:
            angle = 0 - (90 - angle)
        else:
            angle = angle

        M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)
        
        cropped = cv2.warpAffine(cropped, M, size)
        croppedW = H if H > W else W
        croppedH = H if H < W else W
        
        croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW),int(croppedH)), (size[0]/2, size[1]/2))
        return croppedRotated
    return cropped

In [16]:
def detect(img):
    scale_percent = 640/img.shape[1]       
    width = int(img.shape[1] * scale_percent)
    height = int(img.shape[0] * scale_percent)
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    ret, thresh =cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    thresh = cv2.bitwise_not(thresh)
    kernel = np.ones((3, 20), np.uint8)
    thresh = cv2.dilate(thresh, kernel)

    original_sized = cv2.resize(thresh, (img.shape[1],img.shape[0]), interpolation = cv2.INTER_AREA)
    
    contours, hierarchy = cv2.findContours(original_sized,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)    
    
    candidates = []
    index = 0
    added_index = []
    for cnt in contours:
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect) 
        box = np.int0(box)

        cropped = crop_rect(rect,box,img)
        width = cropped.shape[1]
        child_index = hierarchy[0][index][2]
        parent_index = hierarchy[0][index][3]

        if width>95:
            has_overlapped = False
            if child_index in added_index:
                has_overlapped = True
                
            if has_overlapped == False:
                added_index.append(index)
                candidate = {"cropped": cropped, "rect": rect}
                candidates.append(candidate)
        index = index + 1
    return candidates

In [17]:
def decode_image(image):
    result_dict = {}
    results = []        
    
    candidates = detect(image)
    for i in range(len(candidates)):
        candidate = candidates[i]
        cropped = candidate["cropped"]
        rect = candidate["rect"]
        box = cv2.boxPoints(rect) 
        box = np.int0(box)
        ean13, is_valid, thresh = decode(cropped)
        if is_valid:
            result = {}
            result["barcodeFormat"] = "EAN13"
            result["barcodeText"] = ean13
            result["x1"] = int(box[0][0])
            result["y1"] = int(box[0][1])
            result["x2"] = int(box[1][0])
            result["y2"] = int(box[1][1])
            result["x3"] = int(box[2][0])
            result["y3"] = int(box[2][1])
            result["x4"] = int(box[3][0])
            result["y4"] = int(box[3][1])
            results.append(result)

    result_dict["results"] = results
    return result_dict

In [18]:
image_path = 'images/2003889393001.jpg'

In [19]:
image = cv2.imread(image_path)
result_dict = decode_image(image)
results = result_dict["results"]
text = "No barcode found"
if len(results) > 0:
    for result in results:
        if text == "No barcode found":
            text = "Code: "
        ean13 = result["barcodeText"]
        text = text + ean13 + " "
        cv2.line(image,(result["x1"],result["y1"]),(result["x2"],result["y2"]),(0,255,0),3)
        cv2.line(image,(result["x2"],result["y2"]),(result["x3"],result["y3"]),(0,255,0),3)
        cv2.line(image,(result["x3"],result["y3"]),(result["x4"],result["y4"]),(0,255,0),3)
        cv2.line(image,(result["x4"],result["y4"]),(result["x1"],result["y1"]),(0,255,0),3)
print(text)

Code: 2003889393001 
